## Ipsen perturbation bound - inducing sparsity in the original matrix

In the Ipsen SDP relaxation derivation, we start with the matrix to be perturbed (rank-one given by its factor rank_one_vector), then multiply it on the left and right by a diagonal matrix (inducing sparsity). This is relaxed to a Hadamard product with a PSD matrix S (sparsity-inducing), whose elements are in [0, 1].

We are sparsifying and reducing the magnitude of rank_one_vector, so we need to unit norm again. We would like S to be multiplied by some unknown scalar c, so that the L2 norm in the denominator goes to one. This is still the same problem after squaring the expression, then we square root the relation that we would want to have, and we see that we can use the power cone P_3^(1/2) (that is, if the inequality is effective at bounding ||S||).

In [102]:
using COSMO, LinearAlgebra, Random, CSV, Tables, Plots

In [38]:
data = CSV.File("../NC-Data.csv") |> Tables.matrix;
raw_data = sqrt(data)';
n = size(data, 1);
N = n * n;
# All close.
maximum(abs.(raw_data' * raw_data .- data))

4.884981308350689e-15

In [46]:
selected_data = [3; 83; 85];
opt_inds = [79; 80; 81; 84];
k = 7

7

In [36]:
U = svd(raw_data[:, selected_data]).U[:, 1];
rank_one_vector = raw_data' * U;

In [84]:
residual_data = raw_data .- (U * U' * raw_data);

In [85]:
mapslices(norm, raw_data, dims=1)

1×101 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [86]:
mapslices(norm, residual_data, dims=1)

1×101 Array{Float64,2}:
 0.998223  0.986038  0.846852  0.949473  …  0.958106  0.996008  0.92525

In [94]:
model = COSMO.Model()
psd_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.PsdCone)
nn_c = COSMO.Constraint(Matrix(1.0I, N, N), zeros(N), COSMO.Nonnegatives)
power_c = COSMO.Constraint(
    # Three arguments:
    # x_1 = Tr Svv^T = sum_i sum_j S_{ij} v_i v_j
    # x_2 = k^2
    # x_3 = Tr S (vectorize the identity matrix)
    # Constraint: x1 * x2 >= x3^2
    [vec(rank_one_vector * rank_one_vector')';
     zeros(1, N);
     vec(Matrix(1.0I, n, n))'],
    [0.; k^2; 0],
    COSMO.PowerCone(0.5))
# One row: The diag entry may be zero or nonzero.
row_c = repeat(Matrix(-1.0I, n, n), inner=(1, n))
for i = 1:n
    row_c[i, 1 + (n+1)*(i-1)] = k-1
end
row_c = COSMO.Constraint(row_c, zeros(n), COSMO.ZeroSet(n))
assemble!(
    model,
    zeros(N, N),
    -vec((residual_data' * residual_data) .* (rank_one_vector * rank_one_vector')),
    [psd_c, nn_c, power_c, row_c])

In [95]:
result = COSMO.optimize!(model)

>>> COSMO - Results
Status: 

Solved
Iterations: 1700 (incl. 0 safeguarding iterations)
Optimal Objective: -221.4
Runtime: 9427.66ms
Setup Time: 14.63ms

Avg Iter Time: 5.53ms

In [99]:
maximum(abs.(Diagonal(reshape(result.x, n, n))))

32.82766971202997

In [103]:
heatmap(reshape(result.x, n, n))